In [1]:
import sympy as sm
import sympy.physics.mechanics as me
me.init_vprinting(use_latex='mathjax')

In [2]:
class ReferenceFrame(me.ReferenceFrame):

    def __init__(self, *args, **kwargs):

        kwargs.pop('latexs', None)

        lab = args[0].lower()
        tex = r'\hat{{{}}}_{}'

        super(ReferenceFrame, self).__init__(*args,
                                             latexs=(tex.format(lab, 'x'),
                                                     tex.format(lab, 'y'),
                                                     tex.format(lab, 'z')),
                                             **kwargs)
me.ReferenceFrame = ReferenceFrame

In [3]:
L = sm.symbols('L')
q1, q2, u1, u2 = me.dynamicsymbols('q1, q2, u1, u2')

N = me.ReferenceFrame('N')
R = me.ReferenceFrame('R')

R.orient_axis(N, q2, N.z)

In [4]:
N_v_A = u1*N.x
N_v_A

u₁ n_x

In [5]:
N_w_R = u2*N.z
N_w_R

u₂ n_z

In [6]:
r_A_B = -L*R.x
N_v_B = N_v_A + me.cross(N_w_R, r_A_B)

N_v_B.express(N)

(L⋅u₂⋅sin(q₂) + u₁) n_x + -L⋅u₂⋅cos(q₂) n_y

In [7]:
v_A_1 = N_v_A.diff(u1, N)
v_A_2 = N_v_A.diff(u2, N)

v_A_1, v_A_2

(n_x, 0)

In [8]:
v_B_1 = N_v_B.diff(u1, N)
v_B_2 = N_v_B.diff(u2, N)

v_B_1, v_B_2

(n_x, -L r_y)

In [9]:
w_R_1 = N_w_R.diff(u1, N)
w_R_2 = N_w_R.diff(u2, N)

w_R_1, w_R_2

(0, n_z)

In [10]:
me.partial_velocity((N_v_A, N_v_B, N_w_R), (u1, u2), N)

[[n_x, 0], [n_x, -L r_y], [0, n_z]]

In [11]:
x, y, theta = me.dynamicsymbols('x, y, theta')
ux, uy, utheta = me.dynamicsymbols('u_x, u_y, u_theta')

N = me.ReferenceFrame('N')
A = me.ReferenceFrame('A')

A.orient_axis(N, theta, N.z)

O = me.Point('O')
P = me.Point('P')

P.set_pos(O, x*N.x + y*N.y)

O.set_vel(N, 0)

N_v_P = P.vel(N).express(A).xreplace({x.diff(): ux, y.diff(): uy})
N_v_P

(uₓ⋅cos(θ) + u_y⋅sin(θ)) a_x + (-uₓ⋅sin(θ) + u_y⋅cos(θ)) a_y

In [12]:
fn = N_v_P.dot(A.y)
fn

-uₓ⋅sin(θ) + u_y⋅cos(θ)

In [13]:
ux_sol = sm.solve(fn, ux)[0]
ux_sol

 u_y  
──────
tan(θ)

In [14]:
N_v_P.xreplace({ux: ux_sol}).diff(uy, N).simplify()

  1       
────── a_x
sin(θ)    

In [15]:
An = ux_sol.diff(uy)
An

  1   
──────
tan(θ)

In [16]:
(N_v_P.diff(uy, N) + N_v_P.diff(ux, N)*An).simplify()

  1       
────── a_x
sin(θ)    

In [17]:
l = sm.symbols('l')
q1, q2, u1, u2 = me.dynamicsymbols('q1, q2, u1, u2')

N = me.ReferenceFrame('N')
A = me.ReferenceFrame('A')
B = me.ReferenceFrame('B')

A.orient_axis(N, q1, N.z)
B.orient_axis(N, q2, N.z)

O = me.Point('O')
P1 = me.Point('P1')
P2 = me.Point('P2')

O.set_vel(N, 0)

P1.set_pos(O, -l*A.y)
P2.set_pos(P1, -l*B.y)

P1.v2pt_theory(O, N, A)
P2.v2pt_theory(P1, N, B)

P1.vel(N), P2.vel(N)

(l⋅q₁̇ a_x, l⋅q₁̇ a_x + l⋅q₂̇ b_x)

In [18]:
repl = {q1.diff(): u1, q2.diff(): u2}

N_v_P1 = P1.vel(N).xreplace(repl)
N_v_P2 = P2.vel(N).xreplace(repl)

N_v_P1, N_v_P2

(l⋅u₁ a_x, l⋅u₁ a_x + l⋅u₂ b_x)

In [19]:
v_P1_1 = N_v_P1.diff(u1, N)
v_P1_2 = N_v_P1.diff(u2, N)
v_P2_1 = N_v_P2.diff(u1, N)
v_P2_2 = N_v_P2.diff(u2, N)
v_P1_1, v_P1_2, v_P2_1, v_P2_2

(l a_x, 0, l a_x, l b_x)

In [20]:
T1, T2 = me.dynamicsymbols('T1, T2')
m1, m2, g = sm.symbols('m1, m2, g')

R1 = -m1*g*N.y + T1*A.y - T2*B.y
R1

-g⋅m₁ n_y + T₁ a_y + -T₂ b_y

In [21]:
R2 = -m2*g*N.y + T2*B.y
R2

-g⋅m₂ n_y + T₂ b_y

In [22]:
F1 = me.dot(v_P1_1, R1) + me.dot(v_P2_1, R2)
F1

-g⋅l⋅m₁⋅sin(q₁) - g⋅l⋅m₂⋅sin(q₁)

In [23]:
F2 = me.dot(v_P1_2, R1) + me.dot(v_P2_2, R2)
F2

-g⋅l⋅m₂⋅sin(q₂)

In [24]:
m, g, k, l = sm.symbols('m, g, k, l')
q1, q2, u1, u2 = me.dynamicsymbols('q1, q2, u1, u2')

N = me.ReferenceFrame('N')
A = me.ReferenceFrame('A')
B = me.ReferenceFrame('B')

A.orient_axis(N, q1, N.z)
B.orient_axis(A, q2, A.x)

A.set_ang_vel(N, u1*N.z)
B.set_ang_vel(A, u2*A.x)

O = me.Point('O')
Ao = me.Point('A_O')
Bo = me.Point('B_O')

Ao.set_pos(O, l/2*A.x)
Bo.set_pos(O, l*A.x)

O.set_vel(N, 0)
Ao.v2pt_theory(O, N, A)
Bo.v2pt_theory(O, N, A)

Ao.vel(N), Bo.vel(N), A.ang_vel_in(N), B.ang_vel_in(N)

⎛l⋅u₁                                       ⎞
⎜──── a_y, l⋅u₁ a_y, u₁ n_z, u₂ a_x + u₁ n_z⎟
⎝ 2                                         ⎠

In [25]:
v_Ao_1 = Ao.vel(N).diff(u1, N)
v_Ao_2 = Ao.vel(N).diff(u2, N)
v_Bo_1 = Bo.vel(N).diff(u1, N)
v_Bo_2 = Bo.vel(N).diff(u2, N)

v_Ao_1, v_Ao_2, v_Bo_1, v_Bo_2

⎛l                 ⎞
⎜─ a_y, 0, l a_y, 0⎟
⎝2                 ⎠

In [26]:
w_A_1 = A.ang_vel_in(N).diff(u1, N)
w_A_2 = A.ang_vel_in(N).diff(u2, N)
w_B_1 = B.ang_vel_in(N).diff(u1, N)
w_B_2 = B.ang_vel_in(N).diff(u2, N)

w_A_1, w_A_2, w_B_1, w_B_2

(n_z, 0, n_z, a_x)

In [27]:
R_Ao = m*g*N.x
R_Bo = m*g*N.x

R_Ao, R_Bo

(g⋅m n_x, g⋅m n_x)

In [28]:
T_A = -k*q1*N.z + k*q2*A.x
T_B = -k*q2*A.x

T_A, T_B

(-k⋅q₁ n_z + k⋅q₂ a_x, -k⋅q₂ a_x)

In [29]:
F1_A = v_Ao_1.dot(R_Ao) + w_A_1.dot(T_A)
F1_B = v_Bo_1.dot(R_Bo) + w_B_1.dot(T_B)
F2_A = v_Ao_2.dot(R_Ao) + w_A_2.dot(T_A)
F2_B = v_Bo_2.dot(R_Bo) + w_B_2.dot(T_B)

F1_A, F1_B, F2_A, F2_B

⎛  g⋅l⋅m⋅sin(q₁)                                 ⎞
⎜- ───────────── - k⋅q₁, -g⋅l⋅m⋅sin(q₁), 0, -k⋅q₂⎟
⎝        2                                       ⎠

In [30]:
F1 = F1_A + F1_B
F2 = F2_A + F2_B

Fr = sm.Matrix([F1, F2])
Fr

⎡  3⋅g⋅l⋅m⋅sin(q₁)       ⎤
⎢- ─────────────── - k⋅q₁⎥
⎢         2              ⎥
⎢                        ⎥
⎣         -k⋅q₂          ⎦

In [31]:
m, g, k, l = sm.symbols('m, g, k, l')
q1, q2, u1, u2 = me.dynamicsymbols('q1, q2, u1, u2')

N = me.ReferenceFrame('N')
A = me.ReferenceFrame('A')
B = me.ReferenceFrame('B')

A.orient_axis(N, q1, N.z)
B.orient_axis(A, q2, A.x)

A.set_ang_vel(N, u1*N.z)
B.set_ang_vel(A, u2*A.x)

O = me.Point('O')
Ao = me.Point('A_O')
Bo = me.Point('B_O')

Ao.set_pos(O, l/2*A.x)
Bo.set_pos(O, l*A.x)

O.set_vel(N, 0)
Ao.v2pt_theory(O, N, A)
Bo.v2pt_theory(O, N, A)

v_Ao_1 = Ao.vel(N).diff(u1, N)
v_Ao_2 = Ao.vel(N).diff(u2, N)
v_Bo_1 = Bo.vel(N).diff(u1, N)
v_Bo_2 = Bo.vel(N).diff(u2, N)

w_A_1 = A.ang_vel_in(N).diff(u1, N)
w_A_2 = A.ang_vel_in(N).diff(u2, N)
w_B_1 = B.ang_vel_in(N).diff(u1, N)
w_B_2 = B.ang_vel_in(N).diff(u2, N)

In [32]:
Ao.acc(N), Bo.acc(N)

⎛     2                                       ⎞
⎜-l⋅u₁         l⋅u₁̇           2               ⎟
⎜─────── a_x + ──── a_y, -l⋅u₁  a_x + l⋅u₁̇ a_y⎟
⎝   2           2                             ⎠

In [33]:
A.ang_acc_in(N), B.ang_acc_in(N)

(u₁̇ n_z, u₂̇ a_x + u₁⋅u₂ a_y + u₁̇ n_z)

In [34]:
I = m*l**2/12
I

 2  
l ⋅m
────
 12 

In [35]:
I_A_Ao = I*me.outer(A.y, A.y) + I*me.outer(A.z, A.z)
I_B_Bo = I*me.outer(B.x, B.x) + I*me.outer(B.z, B.z)
I_A_Ao

 2  
l ⋅m
────
 12  a_y⊗a_y +  2  
l ⋅m
────
 12  a_z⊗a_z

In [36]:
I_B_Bo

 2  
l ⋅m
────
 12  b_x⊗b_x +  2  
l ⋅m
────
 12  b_z⊗b_z

In [37]:
Rs_Ao = -m*Ao.acc(N)
Rs_Bo = -m*Bo.acc(N)

Rs_Ao, Rs_Bo

⎛      2                                              ⎞
⎜l⋅m⋅u₁        -l⋅m⋅u₁̇             2                  ⎟
⎜─────── a_x + ──────── a_y, l⋅m⋅u₁  a_x + -l⋅m⋅u₁̇ a_y⎟
⎝   2             2                                   ⎠

In [38]:
Ts_A = -(A.ang_acc_in(N).dot(I_A_Ao) +
         me.cross(A.ang_vel_in(N), I_A_Ao).dot(A.ang_vel_in(N)))
Ts_A

  2          
-l ⋅m⋅u₁̇     
───────── a_z
   12        

In [39]:
Ts_B = -(B.ang_acc_in(N).dot(I_B_Bo) +
         me.cross(B.ang_vel_in(N), I_B_Bo).dot(B.ang_vel_in(N)))
Ts_B

⎛   2     2                    2     ⎞       ⎛   2                             ↪
⎜  l ⋅m⋅u₁ ⋅sin(q₂)⋅cos(q₂)   l ⋅m⋅u₂̇⎟       ⎜  l ⋅m⋅(-u₁⋅u₂⋅sin(q₂) + cos(q₂) ↪
⎜- ──────────────────────── - ───────⎟ b_x + ⎜- ────────────────────────────── ↪
⎝             12                12   ⎠       ⎝                  12             ↪

↪         2                ⎞    
↪ ⋅u₁̇)   l ⋅m⋅u₁⋅u₂⋅sin(q₂)⎟    
↪ ──── + ──────────────────⎟ b_z
↪                12        ⎠    

In [40]:
F1s_A = v_Ao_1.dot(Rs_Ao) + w_A_1.dot(Ts_A)
F1s_B = v_Bo_1.dot(Rs_Bo) + w_B_1.dot(Ts_B)
F2s_A = v_Ao_2.dot(Rs_Ao) + w_A_2.dot(Ts_A)
F2s_B = v_Bo_2.dot(Rs_Bo) + w_B_2.dot(Ts_B)

In [41]:
F1s = F1s_A + F1s_B
F2s = F2s_A + F2s_B

Frs = sm.Matrix([F1s, F2s])
Frs

⎡     2        ⎛   2                                    2                ⎞     ↪
⎢  4⋅l ⋅m⋅u₁̇   ⎜  l ⋅m⋅(-u₁⋅u₂⋅sin(q₂) + cos(q₂)⋅u₁̇)   l ⋅m⋅u₁⋅u₂⋅sin(q₂)⎟     ↪
⎢- ───────── + ⎜- ────────────────────────────────── + ──────────────────⎟⋅cos ↪
⎢      3       ⎝                  12                           12        ⎠     ↪
⎢                                                                              ↪
⎢                         2     2                    2                         ↪
⎢                        l ⋅m⋅u₁ ⋅sin(q₂)⋅cos(q₂)   l ⋅m⋅u₂̇                    ↪
⎢                      - ──────────────────────── - ───────                    ↪
⎣                                   12                12                       ↪

↪     ⎤
↪     ⎥
↪ (q₂)⎥
↪     ⎥
↪     ⎥
↪     ⎥
↪     ⎥
↪     ⎥
↪     ⎦